# The effects of studying on high school students

**Authors:** Nadav Tadelis, Sarah Johnson, Chitwan Kaudan

## Abstract

As students, a large part of our daily life is taken up by studying. Currently we must make informed guesses about how much an extra hour of studying will help our grades. However this is not ideal; when making allocation decisions, imperfect information can become a major issue. Specifically, if imperfect information causes us to incorrectly model the effect of studying on grades, then we make poor decisions about how much of our time we should allocate to studying (effectively resulting in a loss of utility). If we were able to find the 'true' marginal effect of studying on grades, then we could calibrate our inner models in a more accurate way. After fitting a naive OLS, then addressing endogenity by using a 2 stage least squares model, we found that a marginal increase in study time per week can increase final exam scores by 21 percentage points.

## Exploratory Data Analysis

The [data](https://archive.ics.uci.edu/ml/datasets/Student+Performance# "Student Performance Data Set") being used are from the public archive of UCI's machine learning repository and were collected by Paulo Cortez of the University of Minho, Portugal in the 2005 - 2006 academic year. Below is a list of all included variables:

#### Attributes for both student-mat.csv (Math course) and student-por.csv (Portuguese language course) datasets:
1. school - student's school (binary: "GP" - Gabriel Pereira or "MS" - Mousinho da Silveira)
2. sex - student's sex (binary: "F" - female or "M" - male)
3. age - student's age (numeric: from 15 to 22)
4. address - student's home address type (binary: "U" - urban or "R" - rural)
5. famsize - family size (binary: "LE3" - less or equal to 3 or "GT3" - greater than 3)
6. Pstatus - parent's cohabitation status (binary: "T" - living together or "A" - apart)
7. Medu - mother's education (numeric: 0 - none,  1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)
8. Fedu - father's education (numeric: 0 - none,  1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)
9. Mjob - mother's job (nominal: "teacher", "health" care related, civil "services" (e.g. administrative or police), "at_home" or "other")
10. Fjob - father's job (nominal: "teacher", "health" care related, civil "services" (e.g. administrative or police), "at_home" or "other")
11. reason - reason to choose this school (nominal: close to "home", school "reputation", "course" preference or "other")
12. guardian - student's guardian (nominal: "mother", "father" or "other")
13. traveltime - home to school travel time (numeric: 1 - \<15 min., 2 - 15 to 30 min., 3 - 30 min. to 1 hour, or 4 - >1 hour)
14. studytime - weekly study time (numeric: 1 - \<2 hours, 2 - 2 to 5 hours, 3 - 5 to 10 hours, or 4 - >10 hours)
15. failures - number of past class failures (numeric: n if 1\<=n\<3, else 4)
16. schoolsup - extra educational support (binary: yes or no)
17. famsup - family educational support (binary: yes or no)
18. paid - extra paid classes within the course subject (Math or Portuguese) (binary: yes or no)
19. activities - extra-curricular activities (binary: yes or no)
20. nursery - attended nursery school (binary: yes or no)
21. higher - wants to take higher education (binary: yes or no)
22. internet - Internet access at home (binary: yes or no)
23. romantic - with a romantic relationship (binary: yes or no)
24. famrel - quality of family relationships (numeric: from 1 - very bad to 5 - excellent)
25. freetime - free time after school (numeric: from 1 - very low to 5 - very high)
26. goout - going out with friends (numeric: from 1 - very low to 5 - very high)
27. Dalc - workday alcohol consumption (numeric: from 1 - very low to 5 - very high)
28. Walc - weekend alcohol consumption (numeric: from 1 - very low to 5 - very high)
29. health - current health status (numeric: from 1 - very bad to 5 - very good)
30. absences - number of school absences (numeric: from 0 to 93)
31. G1 - first period grade (numeric: from 0 to 20) (Math or Portuguese)
31. G2 - second period grade (numeric: from 0 to 20) (Math or Portuguese)
32. G3 - final grade (numeric: from 0 to 20, output target) (Math or Portuguese)

We grouped related variables and plotted their distributions below:

![](fig/quantvar_hist.png)

One thing to note is that first period grade (G1) and second period grade (G2) have similar distributions, but the final period grades (G3) tend to be higher than the first two. This may be due to some sort of curve/weight applied to the final grade.

While these histograms give us some information about the distributions of these individual variables, they don't help with understanding how these variables interact with our dependent variable G3. So let's look at some violin plots to visualize some of these interactions.

![](fig/distrbyschool.png)

These plots suggest some interesting relationships. For example, looking at the `studytime` plot, we see that as the final test scores increase, the distribution of studying shifts to higher levels of studying, but not as dramatically  as one might expect. Additionally, there is a nice relationship between absences and `G3`; the higher scoring students seem to be absent less often, and since absences and study time should be fairly independent<sup>1</sup> we might be able to use `absences` as a measure of ability (or individual preferences about grades).

The differences in the two schools' distributions across variables might be explained by the fact that the Portugal school system implements specialization at the secondary level. High schools in Portugal are concentrated in particular subject like theater or math. These differences are especially noticeable in the last two plots; this suggests that the individual tests may have focused on topics that the schools taught differently. It seems that the G1 and G2 tests were better recieved in `GP` and that the final favored `MS`. We will attempt to account for this specialization amongst schools in our regression.

Now that we have established our data are clean we can move on to trying to answer our question regarding the marginal effect of studying on grades.

-----
<sup>1</sup> We need to make the additional assumption that in secondary school (where parents are notified when students are absent), absences are only caused by illnesses and emergencies (which are independent of study time). Without this assumption it would be plausible that students are skipping school because they value leisure over studying, implying a negative correlation between study time and absences.

## Initial Naive OLS fit


The first step is to build a model and make some assumptions to define the relationship between grades and studying. Let an individual's grade be $\mathrm{G}_i$ and weekly hours of studying be $\mathrm{S}_i$ and their "ability" be $\mathrm{A}_i$. Then we can write:
$$
\mathrm{G}_i = \beta_0 + \beta_1 \mathrm{S}_i + \beta_2 \mathrm{A}_i +\mathrm{U}_i
$$
Where $\mathrm{U}_i$ captures some unobserved heterogeneity and disturbances. 

Clearly, there are issues with this model. How are we defining grades? The ideal set up would have a course specific system of equations, where the number of equations is equal to the number of courses. The next best set up would involve estimating one equation for each type of course (quantitative, literary, historical, etc.). Another alternative would be to define grades as cumulative GPA. In this paper, due to the limitations of the data, we will only be estimating one of the equations described in the second set up; we will define $\mathrm{G}_i$ as the final grade in a Portuguese class. 

Another issue with this model is: how are we defining ability? By its very nature, ability is unobserved. We can proxy for ability using other individual level characteristics (intelligence, age, parents' education, etc.) however we cannot fully capture ability because it is hard to fully define it. Hence, we must keep in mind that the model is not going to be fully specified.

We can think of our utility as being some function of grades, free time (let's consider everything that is not studying, sleeping, or class as "free time"), and studying time. We would expect that the coefficients on grades and free time would be positive, and the coefficient on studying time would be negative, with magnitudes of these coefficients being determined by an individual's preferences. For example, somebody who cares very little about grades, enjoys constantly partying, and hates studying, would have a small positive coefficient on grades, a large positive coefficient on free time, and a large negative coefficient on studying time<sup>1</sup>. This person would maximize their utility and choose how to allocate their time, and they would probably end up spending very little time studying. Notice that before maximizing their utility, an individual would replace grades with the previously defined model for grades (dependent on studying and ability); so someone who heavily values grades could end up having a positive coefficient on studying after plugging the model for grades into their utility function.

Establishing this utility function gives motivation for including variables that might introduce multicollinearity. For example, weekly amount of free time might not improve our estimate of the marginal effect of studying, and would be collinear with the amount of weekly studying. However, when we think of our observations as realizations of a decision making process that involves utility maximization, we can see that there is a strong argument for including free time in the final model.

-----
<sup>1</sup> Of course, for some people and some subject matters of study, the coefficient on studying time may be positive with a decreasing marginal utility. We simplify the specification here dramatically.

In [5]:
import statsmodels.api as sm
naive_ols = sm.load('results/Naive_OLS.pickle')
print(naive_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                G3_perc   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.317
Method:                 Least Squares   F-statistic:                     648.2
Date:                Mon, 11 Dec 2017   Prob (F-statistic):               0.00
Time:                        14:59:14   Log-Likelihood:                 69.417
No. Observations:                 649   AIC:                            -80.83
Df Residuals:                     620   BIC:                             48.95
Df Model:                          28                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4336      0.121      3.572      0.0

These results suggest that within this population, an additional hour of studying per week would "result" in an increase of 3 percentage points to the final grade (with high statistical significance). But this conclusion is not necessarily the correct one.

Many of the coefficients are not statistically significant, and our $X$ matrix is close to singular (suggesting a problem with multicollinearity as `statsmodels.api` explains). While this is not necessarily a cause for concern, it might cast some suspicion on the results.

This result is unsurprising because in this regression we include proxies to represent unobserved ability, and include our variable of interest, hours of studying. This set up makes the implicit assumption that hours of studying and ability are independent. Unfortunately, the validity of this assumption is questionable. It is certainly plausible that students with higher abilities enjoy studying more (perhaps because they find it less difficult), suggesting that they will have a less negative coefficient on studying in their utility maximizations, leading them to study for longer. So the assumption that hours of studying and ability are independent is quite strong, and not necessarily valid. Violation of this assumption would suggest that the dependence between ability and studying is a source of serious multicollinearity. On the other hand, our low $\mathrm{R}^2$ suggests that multicollinearity might not be an issue here; and that we probably just need more data. We could try to address this potential multicollinearity by dropping some variables, but generally it is better to report the full model and point out the potential issues. We don't usually drop variables because that could cause a more egregious problem through confounding effects. The ideal solution here would be to collect more data, but that is not a possibility. While multicollinearity might be present, we can't do much about it.

There is another, more serious, issue with this naive approach of running an OLS fit on the data - endogeneity. In our model we have attempted to use proxies to identify ability, but what if our proxies are not fully identifying ability?  Then clearly the covariance between our $x_k$'s and the error terms is non-zero: we have endogenous variables. This endogeneity means that OLS is not a consistent estimator for our vector of $\beta$'s! We have the following:
$$
\mathrm{b}_k \stackrel{p}{\longrightarrow} \beta_k + \gamma \frac{\mathrm{Cov}[q, x_k]}{\mathrm{Var}[x_k]}
$$
Where $q$ is the unidentified portion of ability, $\gamma$ is the true coefficient on $q$ (if we could identify it), $\mathrm{b}_k$ is the OLS estimated coefficient on $x_k$, and $\beta_k$ is the true coefficient on $x_k$. Notice that $q$ is determined by how well our proxies identify ability. The better our proxies identify ability, the smaller the bias term $\gamma \frac{\mathrm{Cov}[q, x_k]}{\mathrm{Var}[x_k]}$ will be (with bias arising from our endogeneity problem).

In many settings, researchers assume that $\mathrm{Cov}[q, x_k] = 0$ except for the variable of interest (in our case, hours of studying). This is also a plausible assumption in our setting; even more so when we consider that we already determined that study time and ability might be dependent (and that part of ability is in the error term). So we have simplified our problem to identification of the true coefficient on the endogenous variable hours of studying. We should note that since we do not know what part of "ability" is being omitted, we cannot make any guesses about which way our OLS estimated coefficient is biased (because we can't determine the sign of $\mathrm{Cov}[q, studying]$ ).

## Addressing Endogeneity with 2 Stage Least Squares

Now that we have identified our problem, we need to find a way to account for it. Specifically, we need to find one or more instruments for hours of studying. A good instrument, $z$, would have the following characteristics:
* $\mathrm{Cov}[z, studying] \ne 0 \ \quad$ (hours of studying covaries with $z$)
* $\mathrm{Cov}[z, \varepsilon] = 0 \ \ \ \quad\quad\quad$ ($z$ is exogenous)

Finding valid instruments proves to be fairly challenging, because we can often imagine situations where a potential instrument is not valid. For example, we might be tempted to include `school_sup` (extra educational support) as an instrument because if someone is enrolled in extra support they will study more. But upon further thought, it seems likely that enrollment in tutoring is not independent of ability (perhaps high or low ability students are more likely to get tutored), meaning that perhaps `school_sup` covaries with the unidentified ability<sup>1</sup>.

After some thought we concluded that `school_GP` (which school the student is enrolled in), `goout` (how often the student goes out with friends), and `male` (indicator for male), are valid instruments for hours of studying. School enrollment can be a valid instrument because studying habits are often modelled in networks<sup>2</sup>; when one student with high centrality increases their studying, connected students (friends or classmates) also increase their studying. It is easy to imagine that the two schools have friendship networks that have fewer heterophilic links than homophilic links, which implies that hours of studying covaries with enrollment through random perturbations in central students' studying habits. Frequency of going out is a valid instrument because it seems likely that most students go out with friends, and that students who go out more have less time to study. We can assume that going out and ability are independent because there is no reason to think that going out would covary with ability. Lastly, gender is a valid instrument for a similar reason to that of enrollment; male to male links (friendships) will be more common than male to female links, so if central males study less than central females, then we would expect males to study less than females on average (and vice versa).

Additionally we include a new artificial instrument which is derived from the two way frequency table we create in the data exploration notebook. We noticed that there were 149 students that chose their school based on it's proximity to their home, and that 49 of these students had a commute of more than 15 minutes - we create an indicator that indicates whether the observations is one of these 49 students. We argue that this is a valid instrument because those students have chosen a school based on its proximity, and their commute is fairly significant; taking time out of their day that could have been spent studying. We believe that the decrease in available study time arising from the increased commute is a decent instrument in our model.

Now that we have some plausible instruments for our endogenous variable, we can run 2SLS to estimate the model. Note that we exclude `no_parent` (trying to address some multicollinearity from splitting categorical variables into multiple indicators) from the exogenous variables. We are again using a heteroskedasticity robust variance-covariance matrix estimator - because robustness can never hurt.

-----
<sup>1</sup> Earlier we had assumed that all variables other than study time were exogenous, meaning that any of those variables that was correlated with study time would be a valid instrument. But the assumption we made was very weak--in reality we don't really care if variables other than study time are endogenous because we are not interested in the coefficient on these variables. So when we are thinking about constructing instruments for schooling we still need to critically consider the exogeneity of the instrument.

<sup>2</sup> This specific example is from Bryan Graham's course Econ 142 during a discussion of networks and centrality. 

In [6]:
import pickle
with open('results/2SLS.pkl','rb') as f:
    model_results = pickle.load(f)

print(model_results.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                G3_perc   R-squared:                      0.0431
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0031
No. Observations:                 649   F-statistic:                    232.05
Date:                Mon, Dec 11 2017   P-value (F-stat)                0.0000
Time:                        14:29:04   Distribution:                 chi2(26)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          0.4534     0.2234     2.0293     0.0424      0.0155      0.8914
age            0.0005     0.0119     0.0443     0.96

These results are very encouraging. Usually we expect the significance of the coefficients to decrease for almost all the variables in a 2SLS model compared to the analog OLS model. In our 2SLS fit we find that most of the coefficients are indeed less significant, but that the coefficient on study time still has a fairly low variance - and if our instruments are valid, this coefficient should be decent (given the model's limitations). Additionally, the coefficient itself is an order of magnitude larger (0.21 vs 0.03)! This suggests that we have fairly good instruments, and better identification of the marginal effect of studying on grades. Although it is worth pointing out that the assumptions we build our model upon (linearity for example) are not all that rigorous - it is far more likely that studying returns decreasing marginal effects, and we do not account for that possibility, nor the possibility of any interactions. 

One item to note is that in the 2SLS fit, our  $R^2$ value is very small. It turns out that in 2SLS (and 3SLS), a small $R^2$ is not usually a cause for concern. In instrumental variable models we are more interested in the standard error and significance level on the coefficient of our endogenous variable. And in our fit, the coefficient is significant.

## Conclusions

After addressing our issues of endogeneity, we end up with a model that is semi-plausibly identifying the marginal effect of studying on grades. Our results estimate that a marginal increase in hours of studying (per week) results in a 21 point increase in final grade (as a percent). We must keep in mind the limitations of this model; the unrepresentative sample greatly reduces our ability to make general statements about this finding. However, our results are still indicative that in secondary schools, additional studying causes significant increases to final grades. 

A direction for future study would include an increase to the complexity of the model. It seems unlikely that five hours of studying per week would be enough to ensure the 'average student' gets a perfect grade - so perhaps we need to allow for a quadratic (or higher degree) relationship between studying and grades and interactions between studying and other covariates.

### References

Card, D., & Krueger, A. (1992). Does School Quality Matter? Returns to Education and the Characteristics of Public Schools in the United States. *Journal of Political Economy*, 100(1), 1-40.

Cortez, P. and Silva, A. Using Data Mining to Predict Secondary School Student Performance. In A. Brito and J. Teixeira Eds., Proceedings of 5th FUture BUsiness TEChnology Conference (FUBUTEC 2008) pp. 5-12, Porto, Portugal, April, 2008, EUROSIS, ISBN 978-9077381-39-7. 

Greene, W. H. (2000). Econometric analysis. Upper Saddle River, N.J: Prentice Hall.

MacKinnon, J.G. and H. White. (1985), Some heteroskedasticity consistent covariance matrix estimators with improved finite sample properties. *Journal of Econometrics*, 29, 53-57.

### Author Contributions
* Nadav Tadelis: Had idea from a project he did in Econ 152, worked to pick right instruments to improve the 2SLS model, and wrote analysis in main.ipynb.
* Sarah Johnson: Helped brainstorm instruments to improve 2SLS, wrote analysis in main.ipynb, created functions, and integrated with Travis.
* Chitwan Kaudan: Helped brainstorm instruments to improve 2SLS, wrote analysis in main.ipynb, worked on reproducibility aspects, and structured notebooks.